<a href="https://colab.research.google.com/github/smartbin2021/Smart-Bin/blob/main/Senior_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-ocr

     |████████████████████████████████| 61kB 1.7MB/s 
     |████████████████████████████████| 9.5MB 3.6MB/s 
     |████████████████████████████████| 860kB 29.1MB/s 
     |████████████████████████████████| 133kB 35.8MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.6-cp37-none-any.whl size=42937 sha256=fd6c25f9293ef4cf73c0894fbc01146b5c2a980f23bb955c2e809a3a66db8ddc
  Stored in directory: /root/.cache/pip/wheels/79/59/e6/79dd15333b911d51ea68d7e4c325e14f042eacc141201dca44
Successfully built keras-ocr
     |████████████████████████████████| 63.2MB 46kB/s 


In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import keras_ocr
import easyocr
import tensorflow as tf
import tensorflow_hub as hub

 
# show an image
import PIL
from PIL import ImageDraw
from PIL import Image
from numpy import asarray

In [ ]:
remove_dict = {ord('['): None,
               ord(']'): None,
               ord('/'): None,
               ord('\\'):None,
               ord('*'): None,
               ord('$'): None,
               ord('&'): None,
               ord('!'): None,
               ord('@'): None,
               ord('#'): None,
               ord('%'): None,
               ord('^'): None,
               ord('('): None,
               ord(')'): None,
               ord('.'): None,
               ord(','): None,
               ord(';'): None,
               ord(':'): None,
               ord('<'): None,
               ord('>'): None,
               ord('?'): None,
               ord('~'): None}

word_dict = {'brand_rec':['pepsi', 'lumiere','gardenia','almaza','nutella','chtoura','puidor','rim','aquafina','johnnie','laziza',
                          'beirut','dollys','sunbay','pril','dimex','poppins','lipton','perrier','walker','aquafina',
                          'روابي','ريم'],
             
             'brand_non_rec':['master','cheetos','ringo','lays','pringles','dolsi','snickers', 'mars', 'kitkat','bounty','flake',
                              'halabi','twix','bubbly','flake'],

             'brand_limit':['nestle','milka','maccaw','plein','soleil','siblou','americana','alwadi','alakhdar','maxims',
                            'dairy','yamama','aruba',
                            'أروبا','غندور'],

             'word_limit':['coffee','cafe','vinegar','kinder','coco','lait','hot','chili','paste',
                           'خل','قهوة','حار','فلفل','معجون'],

             'word_rec_match':['white','red','pops','mate','tomato','life','fish','fillets',
                               'أحمر','أبيض','الطماطم'],

             'word_nonrec_match':['bueno','orange','melk','frozen','milk','chunky','leo','shrimps','rings'],

             'word_rec':['beer', 'water','sardines','tuna','mayonnaise', 'ketchup','mustard', 'sanitizer','disinfectant','dip','aluminum',
                         'jam','whisky','wine','vodka','flakes','mushrooms','peas','hommus','sauce','scotch','cereal', 'beverage', 'halawa','tea',
                         'حمص','شاي','كاكاو','طحينة','جلاب','حلاوة','مربى','مياه','ماء','مدمس','التونا','كاتشب','معقم','ألمنيوم',],

             'word_non_rec':['bag', 'chips','potato','wipes','soup','leo','soup','bar',
                             'البطاطا','شوربة']} 


In [ ]:
path = "/content/drive/MyDrive/Waste_data/sp_ocr_images/all"
dir_list = [path+'/'+img for img in os.listdir(path) if img.split(".")[-1]=='jpg']
print(len(dir_list))
dir_list

133


['/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp133.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp132.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp131.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp130.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp129.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp128.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp127.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp126.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp125.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp124.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp123.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp122.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp121.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp120.jpg',
 '/content/drive/MyDrive/Waste_data/sp_ocr_images/all/sp119.jpg',
 '/content

In [ ]:
def process_image_ocr(image_path):
  """
  Takes an image file path and returns a list of the image in 4 different rotations as numpy array.
  """
  images_list = []
  angle = 0
  for rotations in range(4):
    # Read in an image file
    image = Image.open(image_path)
    #Rotate image
    image = image.rotate(angle)
    # Turn the jpeg image into numerical numpy array
    image = asarray(image)
    #Update angle
    angle+=90
    #Append image to images list
    images_list.append(image)
  #Read images using the keras-ocr tool
  #images = [keras_ocr.tools.read(img) for img in images_list]
  return images

In [ ]:
#
images = process_image_ocr(dir_list[0])
en_bounds = []
ar_bounds = []
en_b = []
ar_b = []
for image in images:
  en_bounds += reader.readtext(images[0])
  en_b += [b[-2].translate(remove_dict).lower().split(" ") for b in en_bounds]
  

=
#Defining empty list which contains the extracted words
preds = []
#Transforming easyocr bounds nested lists into one list which is preds
for word_list in easy_bounds:
  for word in word_list:
    if word in [''," "]:
      continue
    else:
      preds.append(word)
preds

['804',
 'اح٥ا6ع8',
 '٤ح',
 'س٥',
 '٤٥٤20٤',
 '2م}',
 'ز',
 '٥دة6',
 'ف38',
 'ام6',
 '٥2ح',
 '"8٥',
 'ط',
 'ع2',
 'عىمم|',
 '٨8ص',
 'ع6٥',
 '٨',
 'ظ٥٨=',
 'ز',
 'طنط',
 'لص٢س',
 'ن4',
 'ىم4',
 'ز',
 'مى',
 'م١د',
 'ف4',
 'صس',
 '408',
 '«',
 'س٥',
 'مة8',
 '20',
 'ع',
 '5-',
 '804',
 'اح٥ا6ع8',
 '٤ح',
 'س٥',
 '٤٥٤20٤',
 '2م}',
 'ز',
 '٥دة6',
 'ف38',
 'ام6',
 '٥2ح',
 '"8٥',
 'ط',
 'ع2',
 'عىمم|',
 '٨8ص',
 'ع6٥',
 '٨',
 'ظ٥٨=',
 'ز',
 'طنط',
 'لص٢س',
 'ن4',
 'ىم4',
 'ز',
 'مى',
 'م١د',
 'ف4',
 'صس',
 '408',
 '«',
 'س٥',
 'مة8',
 '20',
 'ع',
 '5-',
 '804',
 'اح٥ا6ع8',
 '٤ح',
 'س٥',
 '٤٥٤20٤',
 '2م}',
 'ز',
 '٥دة6',
 'ف38',
 'ام6',
 '٥2ح',
 '"8٥',
 'ط',
 'ع2',
 'عىمم|',
 '٨8ص',
 'ع6٥',
 '٨',
 'ظ٥٨=',
 'ز',
 'طنط',
 'لص٢س',
 'ن4',
 'ىم4',
 'ز',
 'مى',
 'م١د',
 'ف4',
 'صس',
 '408',
 '«',
 'س٥',
 'مة8',
 '20',
 'ع',
 '5-',
 '804',
 'اح٥ا6ع8',
 '٤ح',
 'س٥',
 '٤٥٤20٤',
 '2م}',
 'ز',
 '٥دة6',
 'ف38',
 'ام6',
 '٥2ح',
 '"8٥',
 'ط',
 'ع2',
 'عىمم|',
 '٨8ص',
 'ع6٥',
 '٨',
 'ظ٥٨=',
 'ز',
 'طنط',
 'لص

In [ ]:
# Create a function to load a trained model
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path, custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [ ]:
classif_model = load_model('/content/drive/MyDrive/Waste_data_models/20210521-sp2-Cnn_Model__transfer_535.h5')

Loading saved model from: /content/drive/MyDrive/Waste_data_models/20210521-sp2-Cnn_Model__transfer_535.h5


In [ ]:
# Define image size
IMG_SIZE = 224

# Create a function for preprocessing images
def process_image(image_path, img_size=IMG_SIZE):
  """
  Takes an image file path and turns the image into a Tensor.
  """
  # Read in an image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired value "IMG_SIZE"
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  #Expand dimensions
  image = np.expand_dims(image,axis=0)
  return image

In [ ]:
def get_predictions(img_path):
  en_bounds = []
  ar_bounds = []
  en_b = []
  ar_b = []
  #Extracting images from file paths for Keras-OCR and EasyOCR
  images = process_image_ocr(img_path)
  #Extracting english text using keras-ocr
  keras_images = [keras_ocr.tools.read(img) for img in images]
  keras_preds = pipeline.recognize(keras_images)
    
  #Defining empty list which contains the extracted words
  preds = []

  #Transforming keras-ocr bounds nested lists into one list which is preds
  for pred in keras_preds:
    for word in pred:
      preds.append(word[0].lower())
  

  #Remove duplicates from prediction list   
  preds = list(set(preds))
  return preds

In [ ]:
preds = get_predictions(dir_list[0])
preds 

['',
 'jsow',
 'in',
 'bio',
 'golden',
 'filtri',
 'leaf',
 'عىمم',
 'alto',
 'upnod',
 'س٥',
 'foglia',
 'ofje',
 'selezione',
 'uobec',
 'eal',
 'ojeidal',
 '408',
 'l',
 'nuepissonue',
 'y',
 'ز',
 'antiossidanti',
 'vnviivli',
 'm',
 '٤ح',
 '5-',
 'ن4',
 'tm',
 'contenuto',
 'naturally',
 'biolea',
 'م١د',
 'yellow',
 's',
 'd',
 'di',
 'e1801',
 'eeioia',
 'صس',
 'g',
 'اح٥ا6ع8',
 '804',
 'oman',
 '3',
 'mollan',
 'loose',
 'onueuod',
 'طنط',
 'ف4',
 'italiana',
 'f',
 'ع',
 '«',
 'olsoc',
 'ف38',
 '2م}',
 '4ogr',
 'enoizies',
 'oaoc',
 'ajenien',
 'ع2|',
 'wl',
 'te',
 '٨ف8ص',
 'ybiy',
 'netto',
 'مى',
 'i',
 'pregiati',
 'c',
 'llvididd',
 'naturalmente',
 '0',
 't',
 'uapiod',
 '11',
 '40g',
 '٥دة6',
 'enwn',
 'jeat',
 'aogr',
 'squepixonue',
 'nw',
 'aquawjenen',
 'ans',
 'e',
 'tea',
 'sov',
 'ام6',
 'pouch',
 'ad',
 '8',
 '٨ظ٥٨=',
 'asoo1',
 'ىم4',
 '=ص٢س',
 'مة8',
 '٥2ح',
 'antioxidants',
 'dragon',
 'high',
 '5',
 'pe',
 '20',
 'ul',
 '٤٥٤20٤',
 '"8٥حط',
 'ip',
 'ةعا6٥',


In [ ]:
def predict(img_path):
  #Get predictions from models 
  preds = get_predictions(img_path)

  #See if the words in pred match any of our pre-defined dictionary words
  result = None
  limit_word = None
  for word in preds: 
    if (word in word_dict["brand_rec"]) or (word in word_dict["word_rec"]):
      result = True
      return result, word
    elif (word in word_dict["brand_non_rec"]) or (word in word_dict["word_non_rec"]):
      result = False
      return result, word
  for word in preds:
    if (word in word_dict["brand_limit"]) or (word in word_dict["word_limit"]):
      limit_word = word
      for match in preds:
        if match in word_dict["word_rec_match"]:
          result = True
          word = word + ', ' +match
          return result, word
        elif match in word_dict["word_nonrec_match"]:
          word = word + ', ' +match
          result = False
          return result, word
  classif_pred = classif_model.predict(process_image(img_path, img_size=IMG_SIZE))
  if classif_pred<=0.5:
    result = True
  else: 
    result = False
  limit_word = "Classifier"

  return result, limit_word

In [ ]:
def plot_prediction(img_path):
  new_path = img_path.split("/")[-1]
  new_path = new_path.split(".")[0]
  new_path = "/content/results/"+ new_path +"_result.png"
  result, word = predict(img_path)
  txt = ""
  # 0 is for recyclable, 1 is for non_recyclable, 2 is for unknown, 3 is for limited
  product_type= None
  if result==True:
    txt = "Recyclable: " + word
    product_type=0
  elif result==False:
    txt = "Non-recyclable: " + word
    product_type=1
  elif result==None and word!=None:
    txt = "Unknown: "+ word
    product_type=2
  else:
    txt = "Unknown: No word match"
    product_type=3
  img = mpimg.imread(img_path)
  imgplot = plt.imshow(img)
  plt.text(0, -20, txt)
  plt.axis('off')
  plt.savefig(new_path, bbox_inches='tight')
  plt.show()
  return product_type

In [ ]:
rec_count = 0
non_rec_count = 0
unknown_count = 0
limited_count = 0
for img_path in dir_list:
  product_type=plot_prediction(img_path)
  if product_type==0:
    rec_count +=1
  elif product_type==1:
    non_rec_count +=1
  elif product_type==2:
    unknown_count +=1
  else:
    limited_count +=1

print("Recyclable: ", rec_count)
print("Non-Recyclable: ", non_rec_count)
print("Unknown: ", unknown_count)
print("Limited: ", limited_count)

In [ ]:
img_path = '/content/sp50.jpg'
preds = get_predictions(img_path)
preds

['٨',
 '',
 'jam',
 'asls',
 'in',
 '٤؟٧٨١',
 'tn',
 '٧،',
 '164',
 'l',
 'wjam',
 'nel',
 '١٥٥',
 '8uly',
 'g',
 'ك٥',
 'uraise',
 'wt',
 'genencally',
 '9',
 '450',
 'clmetcnlly',
 'net',
 'yamama',
 'raisce',
 '٧٢١',
 '٥٥٠',
 '١٥١،',
 '16',
 '116o1',
 'wn',
 'oes',
 'etaen',
 'يمامة',
 '٤']

In [ ]:
  !zip -r /content/ocr_results_may_22_2021.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/sp65_result.png (deflated 0%)
  adding: content/results/sp83_result.png (deflated 0%)
  adding: content/results/sp46_result.png (deflated 0%)
  adding: content/results/sp79_result.png (deflated 0%)
  adding: content/results/sp112_result.png (deflated 0%)
  adding: content/results/sp132_result.png (deflated 0%)
  adding: content/results/sp52_result.png (deflated 0%)
  adding: content/results/sp30_result.png (deflated 0%)
  adding: content/results/sp90_result.png (deflated 0%)
  adding: content/results/sp121_result.png (deflated 0%)
  adding: content/results/sp108_result.png (deflated 0%)
  adding: content/results/sp113_result.png (deflated 0%)
  adding: content/results/sp3_result.png (deflated 0%)
  adding: content/results/sp68_result.png (deflated 0%)
  adding: content/results/sp16_result.png (deflated 0%)
  adding: content/results/sp77_result.png (deflated 0%)
  adding: content/results/sp70_result.png (deflated 0%)
  add